In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
data_train_ori = pd.read_csv('../input/train.csv', header=0)
data_test_ori = pd.read_csv('../input/test.csv', header=0)
# data_train
test_price = pd.read_csv('../input/sample_submission.csv', header=0)
# test_price


# loss function


In [ ]:
def compute_error(a,b,data):
    totalError = 0
    x = data[:,:-1]
    y = data[:,-1]
    s = np.sum(b*x,axis=1)
    totalError = (y-a-s)**2   # 求离差    
    totalError = np.sum(totalError,axis=0)   # 求所有离差的和
    totalError = totalError/float(len(data))
    print('Error:',totalError)
    return totalError#/float(len(data))   # 求离差和的平均值


# Compute gradient

In [ ]:
def compute_gradient(a_current,b_current,data,learning_rate):  #函数compute_gradient 对当前的a,b进行一次梯度迭代运算
    N = float(len(data)*10)
    x = data[:,:-1]    
    y = data[:,-1]
    s = np.sum(b_current*x,axis=1)
   
  
    a_gradient = -(2/N)*(y-s-a_current)
    a_gradient = np.sum(a_gradient,axis=0)  
    cost = (y-s-a_current).reshape(len(data),1)
    b_gradient = -(2/N)*x*cost  #此处求出是正梯度
    b_gradient = np.sum(b_gradient,axis=0)
    
    #a与b值的更新
    new_a = a_current - (learning_rate * a_gradient)  #- (learning_rate * a_gradient) 负梯度
    new_b = b_current - (learning_rate * b_gradient)
        
    return [new_a,new_b]

# PreProcessing

In [ ]:
data_train = data_train_ori[['MSSubClass','LotArea','TotalBsmtSF','1stFlrSF','2ndFlrSF','GrLivArea','GarageArea','SalePrice']]
print (data_train.isnull().sum())
data_train = data_train.values

data_test = data_test_ori[['MSSubClass','LotArea','TotalBsmtSF','1stFlrSF','2ndFlrSF','GrLivArea','GarageArea']]
print(data_test)

#使用SKLearn来填充缺失值
my_imputer = Imputer()
data_imputed = my_imputer.fit_transform(data_test)
df_data_imputed = pd.DataFrame(data_imputed,columns=data_test.columns)
# print (df_data_imputed.isnull().sum())

data_test_y = test_price[['SalePrice']]

data_test = pd.concat((df_data_imputed, data_test_y), axis=1)
data_test = data_test.values
# print(data_test)

#使用SKLearn是数据标准化
ss = StandardScaler()
ss.fit(data_train)
X_train_std = ss.transform(data_train)
X_test_std = ss.transform(data_test)

initial_a = 0.0
initial_b = np.zeros((1,7))
# print(initial_b)
learning_rate = 0.001
num_iter = 3000



In [ ]:

a = initial_a
b = initial_b
for i in range(num_iter):
    a,b =compute_gradient(a,b,X_train_std,learning_rate)
    print('iter {0}:In sample error={1}'.format(i,compute_error(a,b,X_train_std)))

    
print('a = {0}; b = {1}'.format(a,b))
print('Out of sample error={0}'.format(compute_error(a,b,X_test_std)))
